# Import Libraries

In [3]:
%load_ext autoreload
%autoreload 2

In [19]:
import os
import sys
import cv2
import random
import datetime
import neptune
import tempfile
import numpy as np
import pandas as pd

from imutils import paths

import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as prep_input_mobilenetv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


if '../../../notebooks/' not in sys.path:
    sys.path.append('../../../notebooks/')

import utils.constants as cts
import utils.draw_utils as dr

from models.oface_mouth_model import OpenfaceMouth

from data_loaders.data_loader import DLName

from net_data_loaders.net_data_loader import NetDataLoader

# from gt_loaders.gen_gt import Eval
# from gt_loaders.fvc_gt import FVC_GTLoader
# from gt_loaders.pybossa_gt import PybossaGTLoader

# Load the TensorBoard notebook extension
%load_ext tensorboard

## Restrict GPU Memory Growth

In [5]:
## restrict memory growth -------------------

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
try: 
    tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except: 
    raise Exception("Invalid device or cannot modify virtual devices once initialized.")

## restrict memory growth -------------------

# Start Experiment Manager

In [ ]:
print('Starting Neptune')
neptune.init('guilhermemg/icao-nets-training')    
    
def log_data(logs):
    neptune.log_metric('epoch_accuracy', logs['accuracy'])
    neptune.log_metric('epoch_val_accuracy', logs['val_accuracy'])
    neptune.log_metric('epoch_loss', logs['loss'])    
    neptune.log_metric('epoch_val_loss', logs['val_loss']) 

In [7]:
def lr_scheduler(epoch):
    if epoch < 10:
        new_lr = PARAMS['learning_rate']
    else:
        new_lr = PARAMS['learning_rate'] * np.exp(0.1 * ((epoch//50)*50 - epoch))

    #neptune.log_metric('learning_rate', new_lr)
    return new_lr

# Load Data

In [6]:
m = OpenfaceMouth()
req = cts.ICAO_REQ.MOUTH
dl_names = [DLName.FVC_PYBOSSA]
print(f'DL names: {dl_names}')

print('Loading data')
netDataLoader = NetDataLoader(m, req, dl_names, True)
in_data = netDataLoader.load_data()
# in_data = pd.read_csv(cts.LABELS_FQA_SCORES_DATA_PATH)
# in_data = in_data[in_data.fqa_score >= 0.5]
# in_data = in_data.astype({'comp':'str'})
print('Data loaded')

DL names: [<DLName.FVC_PYBOSSA: 'fvc_pybossa'>]
Loading data
Input data.shape: (5780, 4)
Data loaded


In [32]:
in_data.head()

,origin,img_name,comp,aligned
0,fvc_pybossa,/home/guilherme/data/Dropbox/Link to Desktop/D...,1.0,True
1,fvc_pybossa,/home/guilherme/data/Dropbox/Link to Desktop/D...,0.0,True
2,fvc_pybossa,/home/guilherme/data/Dropbox/Link to Desktop/D...,0.0,True
3,fvc_pybossa,/home/guilherme/data/Dropbox/Link to Desktop/D...,0.0,True
4,fvc_pybossa,/home/guilherme/data/Dropbox/Link to Desktop/D...,1.0,True


In [36]:
print(in_data[in_data.comp == '1.0'].shape[0], in_data[in_data.comp == '0.0'].shape[0])

4568 1212


In [43]:
final_df = pd.DataFrame()

df_comp = in_data[in_data.comp == '1.0']
df_non_comp = in_data[in_data.comp == '0.0']

print(df_comp.shape, df_non_comp.shape)

n_imgs = df_non_comp.shape[0]
print(n_imgs)

df_comp = df_comp[:n_imgs].copy()

final_df = final_df.append(df_comp)
final_df = final_df.append(df_non_comp)

print(final_df.shape)
print(final_df[final_df.comp == '1.0'].shape[0])
print(final_df[final_df.comp == '0.0'].shape[0])

(4568, 4) (1212, 4)
1212
(2424, 4)
1212
1212


# Data Generators

In [44]:
TRAIN_PROP = 0.8
VALID_PROP = 0.1
TEST_PROP = 0.1
SEED = 42

print(f'N: {len(in_data)}')

INIT_LR = 1e-4
EPOCHS = 5
BS = 32
SHUFFLE = True
DROPOUT = 0.5
# EARLY_STOPPING = 10
OPTIMIZER = 'Adam'
DENSE_UNITS = 128

print('Starting data generators')
train_valid_df = final_df.sample(frac=TRAIN_PROP+VALID_PROP, random_state=SEED)
test_df = final_df[~final_df.img_name.isin(train_valid_df.img_name)]
# train_valid_df = in_data.sample(frac=TRAIN_PROP+VALID_PROP, random_state=SEED)
# test_df = in_data[~in_data.img_name.isin(train_valid_df.img_name)]

print('Starting data generators')
datagen = ImageDataGenerator(preprocessing_function=prep_input_mobilenetv2, 
                             validation_split=0.2)


train_gen = datagen.flow_from_dataframe(train_valid_df, 
                                        x_col="img_name", 
                                        y_col="comp",
                                        target_size=(224, 224),
                                        class_mode="binary",
                                        batch_size=BS, 
                                        subset='training')

validation_gen = datagen.flow_from_dataframe(train_valid_df,
                                            x_col="img_name", 
                                            y_col="comp",
                                            target_size=(224, 224),
                                            class_mode="binary",
                                            batch_size=BS, 
                                            subset='validation')

test_gen = datagen.flow_from_dataframe(test_df,
                                       x_col="img_name", 
                                       y_col="comp",
                                       target_size=(224, 224),
                                       class_mode="binary",
                                       batch_size=BS)

print(f'TOTAL: {train_gen.n + validation_gen.n + test_gen.n}')

N: 5780
Starting data generators
Starting data generators
Found 1746 validated image filenames belonging to 2 classes.
Found 436 validated image filenames belonging to 2 classes.
Found 242 validated image filenames belonging to 2 classes.
TOTAL: 2424


# Create Experiment

In [45]:
# Define parameters
PARAMS = {'batch_size': BS,
          'n_epochs': EPOCHS,
          'shuffle': SHUFFLE,
          'dense_units': DENSE_UNITS,
          'learning_rate': INIT_LR,
          'optimizer': OPTIMIZER,
          'dropout': DROPOUT,
#           'early_stopping': EARLY_STOPPING,
          'train_prop': TRAIN_PROP,
          'validation_prop': VALID_PROP,
          'test_prop': TEST_PROP,
          'n_train': train_gen.n,
          'n_validation': validation_gen.n,
          'n_test': test_gen.n,
          'seed': SEED}

In [10]:
print('Creating experiment')
neptune.create_experiment(name='train_mobilenetv2',
                          params=PARAMS,
                          properties={
                                      'dl_names': str([dl_name.value for dl_name in dl_names]),
                                      'dl_aligned': True,
                                      'icao_req': req.value,
                                      'tagger_model': m.get_model_name().value},
                          description='Changing learning rate scheduler function to improve results from experiment IC-19',
                          tags=['mobilenetv2'],
                          upload_source_files=['train_mobilenetv2.py']
                         )

Creating experiment
https://ui.neptune.ai/guilhermemg/icao-nets-training/e/IC-20


Experiment(IC-20)

# Training Model

In [46]:
# Clear any logs from previous runs
!rm -rf ./tensorboard_out/ 

In [47]:
class MyCallback(tf.keras.callbacks.Callback): 
    def __init__(self, val_gen):
        self.val_gen = val_gen
        self.out_file_path = 'output/out.csv'
    
    def __clean_out_file(self):
        if os.path.exists(self.out_file_path):
            os.remove(self.out_file_path)
    
    def on_epoch_end(self, epoch, logs={}): 
        
        
        if epoch == 0:
            self.__clean_out_file()
        
        print(f'\nEpoch: {epoch}\n')   
        with open(self.out_file_path,'a') as f:
            predIxs = self.model.predict(self.val_gen, batch_size=BS)
            Y = self.val_gen.labels
            Y_hat = np.argmax(predIxs, axis=1)
            for idx,(y,y_h) in enumerate(zip(Y,Y_hat)):
                if epoch == 0 and idx == 0:
                    f.writelines('epoch,idx,y,y_hat\n')
                f.writelines(f'{epoch},{idx},{y},{y_h}\n')
    
#     def on_batch_end(self, batch, logs={}):
#         print(f'\nbatch: {batch}')
#         print(f'\nlogs: {logs}\n')

In [48]:
print('Training network')

log_dir = "tensorboard_out/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_filepath = '/output/checkpoint_epoch_{}'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq='epoch')

baseModel = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3),
	input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(PARAMS['dense_units'], activation="relu")(headModel)
headModel = Dropout(PARAMS['dropout'])(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
	layer.trainable = False

# compile our model
# opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=PARAMS['optimizer'], metrics=["accuracy"])

# Log model summary
# model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))

# train the head of the network
H = model.fit(
        train_gen,
        steps_per_epoch=train_gen.n // BS,
        validation_data=validation_gen,
        validation_steps=validation_gen.n // BS,
        epochs=EPOCHS,
        callbacks=[
#             LambdaCallback(on_epoch_end = lambda epoch, logs: log_data(logs)),
#             EarlyStopping(patience=PARAMS['early_stopping'], monitor='accuracy', restore_best_weights=True),
              LearningRateScheduler(lr_scheduler),
              MyCallback(validation_gen),
              tensorboard_callback
        ])

Training network
Epoch 1/5
54/54 [==============================] - ETA: 0s - loss: 0.7911 - accuracy: 0.5292
Epoch: 0

54/54 [==============================] - 7s 132ms/step - loss: 0.7911 - accuracy: 0.5292 - val_loss: 0.7040 - val_accuracy: 0.5553
Epoch 2/5
54/54 [==============================] - ETA: 0s - loss: 0.7159 - accuracy: 0.5140
Epoch: 1

54/54 [==============================] - 5s 84ms/step - loss: 0.7159 - accuracy: 0.5140 - val_loss: 0.6978 - val_accuracy: 0.5240
Epoch 3/5
54/54 [==============================] - ETA: 0s - loss: 0.7026 - accuracy: 0.5233
Epoch: 2

54/54 [==============================] - 4s 82ms/step - loss: 0.7026 - accuracy: 0.5233 - val_loss: 0.6957 - val_accuracy: 0.5144
Epoch 4/5
54/54 [==============================] - ETA: 0s - loss: 0.6981 - accuracy: 0.5216
Epoch: 3

54/54 [==============================] - 4s 81ms/step - loss: 0.6981 - accuracy: 0.5216 - val_loss: 0.6946 - val_accuracy: 0.5168
Epoch 5/5
54/54 [==============================] -

# Saving Trained Model

# Test Trained Model

In [31]:
print("Testing Trained Model")

predIdxs = model.predict(test_gen, batch_size=BS)
y_hat = np.argmax(predIdxs, axis=1)

print(classification_report(test_gen.labels, y_hat, target_names=['NON_COMP','COMP']))  
print(accuracy_score(test_gen.labels, y_hat))

Testing Trained Model
              precision    recall  f1-score   support

    NON_COMP       0.19      0.52      0.28       114
        COMP       0.79      0.46      0.58       464

    accuracy                           0.47       578
   macro avg       0.49      0.49      0.43       578
weighted avg       0.67      0.47      0.52       578

0.4688581314878893


# Evaluate Model

In [14]:
print('Evaluating model')
eval_metrics = model.evaluate(test_gen, verbose=0)
for j, metric in enumerate(eval_metrics):
    neptune.log_metric('eval_' + model.metrics_names[j], metric)

Evaluating model


# Finishing Experiment Manager

In [15]:
print('Finishing Neptune')
neptune.stop()

Finishing Neptune
